In [1]:
%load_ext sql
%sql --section duck
import epyfun
sqlite_file = epyfun.get_latest_file("./data/")

Loading configurations from C:\OneDrive\All\strom\pyproject.toml.

Settings changed:

Config,value
feedback,True
autopandas,True
displaycon,False
dsn_filename,./connections.ini


Connecting to 'duck'

In [ ]:
%%sql 

SELECT 
    EXTRACT(YEAR FROM minute) AS year,    
    EXTRACT(MONTH FROM minute) AS month,
    1.0 * AVG(cm) AS cmm,
    1.0 * SUM(cm) AS tot,
    24.0 * 60.0 * cmm AS wd,
    SUM(CASE WHEN value IS NOT NULL THEN 1 ELSE 0 END) AS wobs,
FROM waermestrom_minute
GROUP BY ALL
ORDER BY year, month
;

In [ ]:
%%sql 

x << SELECT *
FROM waermestrom_minute
WHERE minute >= '2024-05-07'
;

In [ ]:
%%sql

SELECT * 
FROM normalstrom
ORDER BY date
;


In [ ]:
%%sql

CREATE OR REPLACE TABLE strom AS
WITH strom_sqlite AS (
    SELECT 
        meterid, 
        -- Blob Functions, because most columns get read as blob
        -- https://duckdb.org/docs/sql/functions/blob
        CAST(decode(date) AS DATETIME) AS date, 
        CAST(decode(value) AS INT) AS value,
        CAST(decode(first) AS INT) AS first
    FROM sqlite_scan('{{sqlite_file}}', 'reading') 
    WHERE meterid = 1 OR meterid = 2 OR meterid = 3
)
SELECT 
    *,
    date_sub(
        'minute', 
        lag(date, 1) OVER(PARTITION BY meterid ORDER BY date),
        date
    ) AS minutes, 
    -- use (1/(1-first)) to induce NA when it is first measurement
    value * (1/(1-first)) - lag(value, 1) OVER(PARTITION BY meterid ORDER BY date) AS consumption,
    1.0 * consumption / minutes AS cm
FROM strom_sqlite
ORDER BY date
;


In [ ]:
%%sql

CREATE OR REPLACE TABLE strom_minute AS
WITH minutes_table AS (
  SELECT UNNEST(generate_series(ts[1], ts[2], interval 1 minute)) AS minute, generate_series AS meterid
  FROM (VALUES (
    [(SELECT MIN(date) FROM strom), (SELECT MAX(date) FROM strom)]
  )) t(ts), generate_series(1, 3)
)
SELECT 
  minutes_table.meterid,
  minutes_table.minute,
  FIRST_VALUE(strom.cm IGNORE NULLS) OVER(
    PARTITION BY minutes_table.meterid 
    ORDER BY minutes_table.minute 
    ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING 
  ) AS cm,
  strom.date,
  strom.value,
  strom.minutes,
  strom.consumption 
FROM minutes_table
LEFT JOIN strom
ON minutes_table.minute = strom.date AND minutes_table.meterid = strom.meterid
ORDER BY minutes_table.minute
;


In [3]:
%%sql

SELECT 
  date,
  "1_cd" AS nd,
  "2_cd" + "3_cd" AS wd,
  "2_cd" AS nt,
  "3_cd" AS ht,
  GREATEST("1_obs", "2_obs", "3_obs") AS obs
FROM (
  WITH cte AS (
    SELECT 
      meterid,
      minute::DATE AS date,
      SUM(cm) AS tot,
      24.0 * 60.0 * AVG(cm) AS cd,
      SUM(CASE WHEN value IS NOT NULL THEN 1 ELSE 0 END) AS obs
    FROM strom_minute
    --WHERE meterid = 1
    GROUP BY ALL
    --HAVING round(tot, 7) != round(cd, 7)
    ORDER BY date
  )
  PIVOT_WIDER cte
  ON meterid
  USING first(cd) AS cd, first(obs) AS obs
  GROUP BY date
)
;


,nd,wd,nt,ht,obs,date,1_cd,1_obs,2_cd,2_obs,3_cd,3_obs
0,6.048074,10.187794,4.874659,5.313135,2.0,2016-07-04,6.048074,2.0,4.874659,2.0,5.313135,2.0
1,4.465727,7.001210,3.278852,3.722357,0.0,2016-07-05,4.465727,0.0,3.278852,0.0,3.722357,0.0
2,4.465727,7.001210,3.278852,3.722357,0.0,2016-07-06,4.465727,0.0,3.278852,0.0,3.722357,0.0
3,4.465727,7.001210,3.278852,3.722357,0.0,2016-07-07,4.465727,0.0,3.278852,0.0,3.722357,0.0
4,4.465727,7.001210,3.278852,3.722357,0.0,2016-07-08,4.465727,0.0,3.278852,0.0,3.722357,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2881,10.919332,6.546488,2.546488,4.000000,2.0,2024-05-24,10.919332,2.0,2.546488,2.0,4.000000,2.0
2882,10.313539,6.011990,6.011990,0.000000,1.0,2024-05-25,10.313539,1.0,6.011990,1.0,0.000000,1.0
2883,13.428571,7.425847,7.425847,0.000000,4.0,2024-05-26,13.428571,4.0,7.425847,4.0,0.000000,4.0
2884,7.178344,5.044586,2.000000,3.044586,4.0,2024-05-27,7.178344,4.0,2.000000,4.0,3.044586,4.0


In [2]:
%%sql

SELECT 
  year, month,
  "1_cd" AS nd,
  "2_cd" + "3_cd" AS wd,
  "2_cd" AS nt,
  "3_cd" AS ht,
  GREATEST("1_obs", "2_obs", "3_obs") AS obs
FROM (
  WITH cte AS (
    SELECT     
      EXTRACT(YEAR FROM minute) AS year,    
      EXTRACT(MONTH FROM minute) AS month,
      * 
    FROM strom_minute
    WHERE 
      (minute >= '2020-12-01' AND minute <= '2021-05-31') OR 
      (minute >= '2022-12-01')
  )
  PIVOT_WIDER cte
  ON meterid
  USING 
    --SUM(cm) AS cd,
    AVG(cm* 24.0 * 60.0 * 30)  AS cd, 
    SUM(CASE WHEN value IS NOT NULL THEN 1 ELSE 0 END) AS obs
  GROUP BY year, month
)
;


,year,month,nd,wd,nt,ht,obs
0,2020,12,295.694648,659.275304,342.808570,316.466734,67.0
1,2021,1,279.743140,723.524955,431.266891,292.258065,96.0
2,2021,2,274.541775,653.529888,361.882110,291.647778,58.0
3,2021,3,262.314802,411.163673,241.190295,169.973378,45.0
4,2021,4,265.068788,330.283409,195.152614,135.130795,35.0
5,2021,5,206.593444,263.714437,158.147918,105.566519,31.0
6,2022,12,303.102365,779.647538,431.494916,348.152622,53.0
7,2023,1,264.062989,624.687027,352.954649,271.732378,82.0
8,2023,2,288.676571,578.205055,301.750153,276.454902,46.0
9,2023,3,252.882620,373.048430,196.807191,176.241239,24.0


In [2]:
%%sql

SELECT     
  EXTRACT(YEAR FROM minute) AS year,    
  EXTRACT(MONTH FROM minute) AS month,
  * 
FROM strom_minute
;

,year,month,meterid,minute,cm,date,value,minutes,consumption
0,2016,7,2,2016-07-04 07:50:00,0.100000,2016-07-04 07:50:00,0.0,NaN,NaN
1,2016,7,1,2016-07-04 07:50:00,0.100000,2016-07-04 07:50:00,0.0,NaN,NaN
2,2016,7,3,2016-07-04 07:50:00,0.100000,2016-07-04 07:50:00,0.0,NaN,NaN
3,2016,7,2,2016-07-04 07:51:00,0.100000,NaT,NaN,NaN,NaN
4,2016,7,1,2016-07-04 07:51:00,0.100000,NaT,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
12469420,2024,5,3,2024-05-29 18:23:00,0.000000,2024-05-29 18:23:00,2371.0,363.0,0.0
12469421,2024,5,1,2024-05-29 18:23:00,0.008242,NaT,NaN,NaN,NaN
12469422,2024,5,2,2024-05-29 18:24:00,NaN,NaT,NaN,NaN,NaN
12469423,2024,5,3,2024-05-29 18:24:00,NaN,NaT,NaN,NaN,NaN
